In [1]:
import pandas as pd
import scipy
import numpy as np
import math

In [28]:
#World Hapiness Data
wh_data = pd.read_csv("https://raw.githubusercontent.com/joannarashid/cse6242_proj/main/WH_data_2019.csv")

#Women's Prosperity Index
wps_data = pd.read_csv("https://raw.githubusercontent.com/joannarashid/cse6242_proj/main/WPS-Index-2021-Data.csv", 
                       encoding = 'unicode_escape')

#Tropical Climate Data
tropical_data = pd.read_csv("https://raw.githubusercontent.com/joannarashid/cse6242_proj/main/tropical_countries.csv")

#climate data
climate_data = pd.read_csv('https://raw.githubusercontent.com/joannarashid/cse6242_proj/main/climate_zones.csv')

#LGBTQ Safety and Welfare Data
lgbtq_data = pd.read_csv("https://raw.githubusercontent.com/joannarashid/cse6242_proj/main/LGBTQ_Safety_Index.csv")

#Industry Sector Data
sector_data = pd.read_csv("https://raw.githubusercontent.com/joannarashid/cse6242_proj/main/country_sectors.csv")

#City Climate Data
city_data = pd.read_csv("https://raw.githubusercontent.com/ereppond/CSE6242-GroupProject/main/BlissfulBorders/data/city_data.csv")

#air quality Data
aq_data = pd.read_csv('https://raw.githubusercontent.com/joannarashid/cse6242_proj/main/SEDAC_annual_pm2-5_concentration.csv')

#air quality Data
aq_data = pd.read_csv('https://raw.githubusercontent.com/joannarashid/cse6242_proj/main/SEDAC_annual_pm2-5_concentration.csv')


In [29]:
#Some column renaming to prepare for joins

#Clean World Happiness Data (preserving all vars)
wh_data.rename(columns = {'Overall rank': 'World Happiness Rank', 
                          'Country or region': 'Country',
                          'Score': 'World Happiness Score'}, inplace = True)

#Clean Women's Prosperity Index Data
wps_data.rename(columns = {'ï»¿WPS Index rank': "Women's Peace & Prosperity Rank", 
                           'WPS Index score': "Women's Peace & Prosperity Score", 
                           'Country':'WPS Country'}, inplace = True)
wps_data = wps_data.iloc[:,0:3] #select vars

#Clean Tropical climate data
climate_data.rename(columns = {'Country':'Climate Country'}, inplace = True)
climate_data['Climate Country'] = climate_data['Climate Country'].str.strip() #removing leading spaces

#Clean LGBTQ data
lgbtq_data.rename(columns = {'Rank': 'LGBTQ Rights Rank', 
                             "Score \n(worst is -100)\n(best is 0)": 'LGBTQ Rights Score',
                             'Grade': 'LGBTQ Rights Grade',
                             'Country': 'LGBTQ Country'}, inplace = True)

lgbtq_data = lgbtq_data[['LGBTQ Rights Rank', 'LGBTQ Country', 'LGBTQ Rights Score', 'LGBTQ Rights Grade']]

#Clean sector data
sector_data.rename(columns = {'Country': 'Sector Country'}, inplace = True)
sector_data['Sector Country'] = sector_data['Sector Country'].str.strip() #removing leading spaces


In [30]:
###Country Level only

#since the World Happiness Index is the objectove value for this application, 
#wh_data is the left df on whihc the df is started which ensures all countries in the WHI are included
#subsequent joins eliminate observations that are not in the WHI

df_country = wh_data.merge(wps_data,
                   how = 'left', 
                   left_on = "Country",
                   right_on = "WPS Country")

#merge LGBTQ data with main df
df_country = df_country.merge(lgbtq_data,
              how = 'left',
              left_on = 'Country',
              right_on = 'LGBTQ Country')

#merge climate type data with main df
df_country = df_country.merge(climate_data,
              how = 'left',
              left_on = 'Country',
              right_on = 'Climate Country')

#merge economic sector data data with main df
df_country = df_country.merge(sector_data,
              how = 'left',
              left_on = 'Country',
              right_on = 'Sector Country')


#drop duplicate columns
df_country.drop(['Climate Country', 'WPS Country', 'Avg_temp_F', 'Avg_temp_C',
       'Sector Country','WPS Country','Sector Country','Sector Country'], axis=1, inplace=True) 



In [31]:
#assign int values to LGBTQ letter grades
grades = list(df_country['LGBTQ Rights Grade'].unique())
grades = [grade for grade in grades if type(grade) == str] #only letter grades
values = sorted(list(range(1,len(grades))), reverse=True) # list of integers in reverse
scores = dict(zip(grades, values))
df_country['LGBTQ Rights Score'] = df_country['LGBTQ Rights Grade'].apply(lambda x: scores.get(x)) #new column with inter values for grades

#Clean LGBTQ grades to remove + and -
df_country['LGBTQ Rights Grade'] = df_country['LGBTQ Rights Grade'].str.replace('+', '')
df_country['LGBTQ Rights Grade'] = df_country['LGBTQ Rights Grade'].str.replace('-', '')

#convert sector data to decimal
df_country['Agricultural percent'] = df_country['Agricultural percent'].str.rstrip('%').astype('float') / 100.0
df_country['Industrial percent'] = df_country['Industrial percent'].str.rstrip('%').astype('float') / 100.0
df_country['Service percent'] = df_country['Service percent'].str.rstrip('%').astype('float') / 100.0

#add dominant sector
df_country["Main Sector"] = df_country[['Agricultural percent','Industrial percent','Service percent']].idxmax(axis=1)
df_country["Main Sector"] = df_country["Main Sector"].str.replace(' percent', '')

#Normalizing data to range 0,1
df_country['LGBTQ_norm'] = (df_country['LGBTQ Rights Score'] - df_country['LGBTQ Rights Score'].min()) / (df_country['LGBTQ Rights Score'].max() - df_country['LGBTQ Rights Score'].min())
df_country['WPS_norm'] = (df_country["Women's Peace & Prosperity Score"] - df_country["Women's Peace & Prosperity Score"].min()) / (df_country["Women's Peace & Prosperity Score"].max() - df_country["Women's Peace & Prosperity Score"].min())
df_country['Freedom_norm'] = (df_country['Freedom to make life choices'] - df_country['Freedom to make life choices'].min()) / (df_country['Freedom to make life choices'].max() - df_country['Freedom to make life choices'].min())
df_country['GDP_norm'] = (df_country['GDP per capita'] - df_country['GDP per capita'].min()) / (df_country['GDP per capita'].max() - df_country['GDP per capita'].min())
df_country['WH_norm'] = (df_country['World Happiness Score'] - df_country['World Happiness Score'].min()) / (df_country['World Happiness Score'].max() - df_country['World Happiness Score'].min())

#additing climate infomation detail
climate_codes = df_country['Climate zone'].unique()

climate_zones = {
                "AF": ["Tropical rainforest", "Tropical"],
                "AM": ["Tropical monsoon", "Tropical"],
                "AW": ["Tropical wet and dry or savanna", "Tropical"],
                "BWH": ["Subtropical desert", "Arid"],
                "BSH": ["Subtropical steppe", "Arid"],
                "BSK": ["Mid-latitude steppe", "Arid"],
                "BWK": ["Mid-latitude desert", "Arid"],
                "CFA": ["Humid subtropical, no dry season", "Temperate"],
                "CWA": ["Humid subtropical, dry winter", "Temperate"],
                "CSA": ["Mediterranean, hot summer", "Temperate"],
                "CSB": ["Mediterranean, warm summer", "Temperate"],
                "CWB": ["Temperate highland tropical climate with dry winters", "Temperate"],
                "DFB": ["Humid continental, no dry season, warm summer", "Cold"],
                "DFC": ["Subartic, severe winter, no dry season, cool summer", "Cold"],
                "DSC": ["Humid continental, dry warm summer", "Cold"],
                "DSB": ["Humid continental, dry warm summer", "Cold"],
                "DWA": ["Humid continental, severe dry winter, hot summer", "Cold"],
                "DWB": ["Humid continental, severe dry winter, warm summer", "Cold"],
                "DWC": ["Subartic, dry winter, cool summer", "Cold"],
                "ET": ["Tundra", "Cold"]
                 }

def map_climate_zones(zone):
    """
    Maps descriptions found in climate_zones dict to climate code in df per dict
    """
    if zone in climate_zones:
        return climate_zones[zone]
    else:
        return ["", ""]

# apply the mapping function to the climate zone column and create two new columns for descriptions
df_country[["Climate description", "Climate type"]] = df_country["Climate zone"].apply(map_climate_zones).tolist() 


<ipython-input-31-e333ae133230>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_country['LGBTQ Rights Grade'] = df_country['LGBTQ Rights Grade'].str.replace('+', '')


In [32]:
def get_grade(score):
    if score >= 0.9:
        return 'A'
    elif score >= 0.8:
        return 'B'
    elif score >= 0.7:
        return 'C'
    elif score >= 0.6:
        return 'D'
    else:
        return 'F'

# Apply the function to the 'score' column to create a new column 'grade'
df_country["Women's Peace and Prosperity Grade"] = df_country['WPS_norm'].apply(get_grade)
df_country["Happiness Grade"] = df_country['WH_norm'].apply(get_grade)

In [35]:
df_country.head()

,World Happiness Rank,Country,World Happiness Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Women's Peace & Prosperity Rank,...,Main Sector,LGBTQ_norm,WPS_norm,Freedom_norm,GDP_norm,WH_norm,Climate description,Climate type,Women's Peace and Prosperity Grade,Happiness Grade
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,2.0,...,Service,1.0,0.979814,0.944532,0.795724,1.000000,"Subartic, severe winter, no dry season, cool s...",Cold,A,A
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,4.0,...,Service,1.0,0.970497,0.938193,0.821259,0.965622,,,A,A
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,1.0,...,Service,0.9,1.000000,0.955626,0.883610,0.956265,"Subartic, severe winter, no dry season, cool s...",Cold,A,A
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,3.0,...,Service,1.0,0.976708,0.936609,0.819477,0.944060,Tundra,Cold,A,A
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,10.0,...,Service,0.9,0.942547,0.882726,0.828979,0.942840,,,A,A


In [36]:
df_country.to_csv('df_countries_only.csv')